# Module Project
## Synthesizing raw audio signals to sound like saxophone sounds

In [8]:
pip install nbformat

  Using cached nbformat-5.10.4-py3-none-any.whl.metadata (3.6 kB)
  Using cached fastjsonschema-2.19.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached jsonschema-4.21.1-py3-none-any.whl.metadata (7.8 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached jsonschema_specifications-2023.12.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached pkgutil_resolve_name-1.3.10-py3-none-any.whl.metadata (624 bytes)
  Using cached referencing-0.34.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached rpds_py-0.18.0-cp38-cp38-macosx_10_12_x86_64.whl.metadata (4.1 kB)
Using cached nbformat-5.10.4-py3-none-any.whl (78 kB)
Using cached fastjsonschema-2.19.1-py3-none-any.whl (23 kB)
Using cached jsonschema-4.21.1-py3-none-any.whl (85 kB)
Using cached attrs-23.2.0-py3-none-any.whl (60 kB)
Using cached jsonschema_specifications-2023.12.1-py3-none-any.whl (18 kB)
Using cached pkgutil_resolve_name-1.3.10-py3-none-any.whl (4.7 kB)
Using cached referencing-0.34.0-py3-none-any.whl (2

In [13]:
import numpy as np
from scipy.io import wavfile
import matplotlib.pyplot as plt
plt.style.use('seaborn-dark')
%run Util.ipynb
%matplotlib inline
import IPython.display as ipd

/var/folders/wz/njd4pyh975g7stkhmhcy2frr0000gn/T/ipykernel_57284/3212850412.py:4: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-dark')


In [12]:
# Get middle C frequency
note_freqs = get_music_notes()
frequency = note_freqs['C4']

# Pure sine wave
sine_wave = get_sine_wave(frequency, duration=2, amplitude=2048)
wavfile.write('data/pure_c.wav', rate=44100, data=sine_wave.astype(np.int16))

In [15]:
ipd.Audio(sine_wave, rate=44100, normalize=True)